In [1]:
:e DataKinds
:e TypeOperators
:e TypeApplications
:e ScopedTypeVariables
:e FlexibleContexts
:e ViewPatterns
:e KindSignatures
:e TypeFamilies
:e RankNTypes
:e GADTs

:s -fplugin GHC.TypeLits.KnownNat.Solver
:s -fplugin GHC.TypeLits.Normalise

import           GHC.TypeLits
import           Numeric.LinearAlgebra.Static
import qualified Numeric.LinearAlgebra.Static as S
import qualified Numeric.LinearAlgebra as LA
import           Data.Proxy
import qualified Numeric.LinearProgramming.L1 as L1
import Data.Maybe
import System.Random hiding (split)
import qualified Data.Vector.Storable as V
import Data.Proxy
import Data.Tuple.HT (fst3)

-- For plotting
import Graphics.Rendering.Chart
import Data.Colour
import Data.Colour.Names
import Data.Default.Class
import Graphics.Rendering.Chart.Backend.Cairo
import Control.Lens
import IHaskellPrelude hiding ((<>))
import Data.List (unfoldr)


In [2]:
data UnitaryRational (n :: Nat) (m :: Nat) where
    UnitaryRational :: (KnownNat n, KnownNat m, 1 <= m, n <= m) => UnitaryRational n m

instance (KnownNat n, KnownNat m) => Show (UnitaryRational n m) where
    show _ = "UnitaryRational " ++ show n' ++ " " ++ show m'
        where
            n' = fromIntegral . natVal $ (undefined :: Proxy n) :: Int
            m' = fromIntegral . natVal $ (undefined :: Proxy m) :: Int

In [3]:
unitary_to_pair :: forall n m. (KnownNat n, KnownNat m) => UnitaryRational n m -> (Integer, Integer)
unitary_to_pair _ = (num, denom)
    where
        num = natVal $ (undefined :: Proxy n) :: Integer
        denom = natVal $ (undefined :: Proxy m) :: Integer
        
unitary_to_fractional :: forall n m o. (KnownNat n, KnownNat m, Fractional o) => UnitaryRational n m -> o
unitary_to_fractional x = (fromIntegral num) / (fromIntegral denom)
    where
        (num, denom) = unitary_to_pair x
        

In [4]:
unitary_to_fractional (UnitaryRational :: UnitaryRational 3 5)

0.6

In [5]:
-- https://en.wikipedia.org/wiki/Karmarkar%27s_algorithm
affine_scaling :: (KnownNat m, KnownNat n, KnownNat p, KnownNat q, 1 <= n, 1 <= m) => L m n -> R m -> R n -> R n -> UnitaryRational p q -> [Maybe (R n)]
affine_scaling a b c x0 gamma = iterate (>>= go) (Just x0)
    where
        gamma' = unitary_to_fractional gamma
        
        min_vh :: (KnownNat n) => R n -> R n -> Double
        min_vh v h = minimum . map fst . filter ((<0) . snd) $ vals_with_h
            where
                vals = zipWithVector (\vi hi -> -vi/hi) v h
                vals_with_h = zip (LA.toList $ extract vals) (LA.toList $ extract h)
                        
        go xk = let
            vk = b - a #> xk
            dm_negsq = diag $ dvmap (**(-2)) vk
            hx = inv ((tr a) S.<> dm_negsq S.<> a) #> c
            hv = ((konst (-1)) * a #> hx)
            alpha = gamma' * min_vh vk hv
            xkp1 = xk + (konst alpha) * hx
            in 
                if ((>=0) . maximum . LA.toList . extract $ hv)
                then Nothing
                else Just xkp1

In [6]:
p = vector [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0] :: R 11

-- min c^Tx
c = vec2 1 1

-- s.t Ax <= b
a = ((konst 2) * (col p)) ||| (1 :: L 11 1)
b = p**2 + 1

In [9]:
lpSol = affine_scaling a b c (konst 0) (UnitaryRational :: UnitaryRational 50 100)

In [10]:
mapM_ (disp 10 . fromJust) . take 20 $ lpSol

R 2
0  0
R 2
0.3834998360  0.2064002625
R 2
0.4760045477  0.4420945756
R 2
0.4928840659  0.5911654957
R 2
0.4970811507  0.6699436301
R 2
0.4985057320  0.7100066584
R 2
0.4991187905  0.7301374047
R 2
0.4994287564  0.7401993412
R 2
0.4995954129  0.7452186359
R 2
0.4996829038  0.7477241206
R 2
0.4997272267  0.7489762855
R 2
0.4997493491  0.7496024070
R 2
0.4997603650  0.7499155130
R 2
0.4997658564  0.7500720827
R 2
0.4997685971  0.7501503724
R 2
0.4997699662  0.7501895186
R 2
0.4997706504  0.7502090920
R 2
0.4997709924  0.7502188788
R 2
0.4997711634  0.7502237722
R 2
0.4997712489  0.7502262189